<a href="https://colab.research.google.com/github/juancgar/memoryAgent/blob/main/AgentResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain_openai

In [ ]:
pip install git+https://github.com/brandonstarxel/chunking_evaluation.git

  Cloning https://github.com/brandonstarxel/chunking_evaluation.git to /tmp/pip-req-build-0kvi4jgh
  Running command git clone --filter=blob:none --quiet https://github.com/brandonstarxel/chunking_evaluation.git /tmp/pip-req-build-0kvi4jgh
  Resolved https://github.com/brandonstarxel/chunking_evaluation.git to commit a503ddf9af2275d492881c4fa6918d99ae37ab50
  Preparing metadata (setup.py) ... done


In [ ]:
pip install pypdf


In [ ]:
pip install langchain_community

In [ ]:
pip install -U weaviate-client

In [ ]:
from langchain_openai import ChatOpenAI

from google.colab import userdata


llm = ChatOpenAI(temperature=0.7, model="gpt-4o", api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# Define System Prompt
system_prompt = SystemMessage("You are a helpful AI Assistant. Answer the User's queries succinctly in one sentence.")

# Start Storage for Historical Message History
messages = [system_prompt]

while True:

    # Get User's Message
    user_message = HumanMessage(input("\nUser: "))

    if user_message.content.lower() == "exit":
        break

    else:
        # Extend Messages List With User Message
        messages.append(user_message)

    # Pass Entire Message Sequence to LLM to Generate Response
    response = llm.invoke(messages)

    print("\nAI Message: ", response.content)

    # Add AI's Response to Message List
    messages.append(response)


User: Hi good night

AI Message:  Good night! How can I assist you today?

User: I want to learn more about marine biology, can you point me to good papers to start investigating about coral reef and the effect of global warming on it?

AI Message:  Certainly! A good starting point is the paper "Coral Reefs Under Rapid Climate Change and Ocean Acidification" by Hoegh-Guldberg et al., published in Science, and "Climate Change, Human Impacts, and the Resilience of Coral Reefs" by Hughes et al., also in Science.

User: can you summarise in 100 words the first paper

AI Message:  The paper "Coral Reefs Under Rapid Climate Change and Ocean Acidification" by Hoegh-Guldberg et al. discusses the vulnerability of coral reefs to climate change and ocean acidification, driven by increased atmospheric CO2 levels. It highlights how rising sea temperatures lead to coral bleaching and mortality, while acidification reduces calcification rates, weakening reef structures. The authors emphasize the cri

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

reflection_prompt_template = """
You are analyzing conversations about research papers to create memories that will help guide future interactions. Your task is to extract key elements that would be most helpful when encountering similar academic discussions in the future.

Review the conversation and create a memory reflection following these rules:

1. For any field where you don't have enough information or the field isn't relevant, use "N/A"
2. Be extremely concise - each string should be one clear, actionable sentence
3. Focus only on information that would be useful for handling similar future conversations
4. Context_tags should be specific enough to match similar situations but general enough to be reusable

Output valid JSON in exactly this format:
{{
    "context_tags": [              // 2-4 keywords that would help identify similar future conversations
        string,                    // Use field-specific terms like "deep_learning", "methodology_question", "results_interpretation"
        ...
    ],
    "conversation_summary": string, // One sentence describing what the conversation accomplished
    "what_worked": string,         // Most effective approach or strategy used in this conversation
    "what_to_avoid": string        // Most important pitfall or ineffective approach to avoid
}}

Examples:
- Good context_tags: ["transformer_architecture", "attention_mechanism", "methodology_comparison"]
- Bad context_tags: ["machine_learning", "paper_discussion", "questions"]

- Good conversation_summary: "Explained how the attention mechanism in the BERT paper differs from traditional transformer architectures"
- Bad conversation_summary: "Discussed a machine learning paper"

- Good what_worked: "Using analogies from matrix multiplication to explain attention score calculations"
- Bad what_worked: "Explained the technical concepts well"

- Good what_to_avoid: "Diving into mathematical formulas before establishing user's familiarity with linear algebra fundamentals"
- Bad what_to_avoid: "Used complicated language"

Additional examples for different research scenarios:

Context tags examples:
- ["experimental_design", "control_groups", "methodology_critique"]
- ["statistical_significance", "p_value_interpretation", "sample_size"]
- ["research_limitations", "future_work", "methodology_gaps"]

Conversation summary examples:
- "Clarified why the paper's cross-validation approach was more robust than traditional hold-out methods"
- "Helped identify potential confounding variables in the study's experimental design"

What worked examples:
- "Breaking down complex statistical concepts using visual analogies and real-world examples"
- "Connecting the paper's methodology to similar approaches in related seminal papers"

What to avoid examples:
- "Assuming familiarity with domain-specific jargon without first checking understanding"
- "Over-focusing on mathematical proofs when the user needed intuitive understanding"

Do not include any text outside the JSON object in your response.

Here is the prior conversation:

{conversation}
"""

reflection_prompt = ChatPromptTemplate.from_template(reflection_prompt_template)

reflect = reflection_prompt | llm | JsonOutputParser()

In [ ]:
def format_conversation(messages):

    # Create an empty list placeholder
    conversation = []

    # Start from index 1 to skip the first system message
    for message in messages[1:]:
        conversation.append(f"{message.type.upper()}: {message.content}")

    # Join with newlines
    return "\n".join(conversation)

conversation = format_conversation(messages)

print(conversation)

HUMAN: Hi good night
AI: Good night! How can I assist you today?
HUMAN: I want to learn more about marine biology, can you point me to good papers to start investigating about coral reef and the effect of global warming on it?
AI: Certainly! A good starting point is the paper "Coral Reefs Under Rapid Climate Change and Ocean Acidification" by Hoegh-Guldberg et al., published in Science, and "Climate Change, Human Impacts, and the Resilience of Coral Reefs" by Hughes et al., also in Science.
HUMAN: can you summarise in 100 words the first paper
AI: The paper "Coral Reefs Under Rapid Climate Change and Ocean Acidification" by Hoegh-Guldberg et al. discusses the vulnerability of coral reefs to climate change and ocean acidification, driven by increased atmospheric CO2 levels. It highlights how rising sea temperatures lead to coral bleaching and mortality, while acidification reduces calcification rates, weakening reef structures. The authors emphasize the critical threshold of a 1-2°C inc

In [ ]:
reflection = reflect.invoke({"conversation": conversation})

print(reflection)

{'context_tags': ['marine_biology', 'coral_reef', 'climate_change_impact'], 'conversation_summary': "Provided a summary of the paper on coral reefs' vulnerability to climate change and ocean acidification.", 'what_worked': 'Recommending well-regarded papers and providing concise summaries upon request.', 'what_to_avoid': 'N/A'}


In [ ]:
import weaviate
from weaviate.classes.init import Auth
import os

# Best practice: store your credentials in environment variables
wcd_url = userdata.get('WCD_UR')
wcd_api_key = userdata.get('WCD_API_KEY')
openai_api_key = userdata.get('OPENAI_API_KEY')

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,  # Replace with your Weaviate Cloud URL
    auth_credentials=Auth.api_key(wcd_api_key),  # Replace with your Weaviate Cloud key
    headers={'X-OpenAI-Api-key': openai_api_key}  # Replace with your OpenAI API key
)

In [ ]:
print("Connected to Weviate: ", client.is_ready())


Connected to Weviate:  True


In [ ]:
from weaviate.classes.config import Property, DataType, Configure, Tokenization

client.collections.delete("episodic_memory")
client.collections.create(
    name="episodic_memory",
    description="Collection containing historical chat interactions and takeaways.",
    vectorizer_config=Configure.Vectorizer.text2vec_openai(), # Change to text2vec-openai for Weaviate Cloud
    properties=[
        Property(name="conversation", data_type=DataType.TEXT),
        Property(name="context_tags", data_type=DataType.TEXT_ARRAY),
        Property(name="conversation_summary", data_type=DataType.TEXT),
        Property(name="what_worked", data_type=DataType.TEXT),
        Property(name="what_to_avoid", data_type=DataType.TEXT),

    ]
)

In [ ]:
def add_episodic_memory(messages, vdb_client):

    # Format Messages
    conversation = format_conversation(messages)

    # Create Reflection
    reflection = reflect.invoke({"conversation": conversation})

    # Load Database Collection
    episodic_memory = vdb_client.collections.get("episodic_memory")

    # Insert Entry Into Collection
    episodic_memory.data.insert({
        "conversation": conversation,
        "context_tags": reflection['context_tags'],
        "conversation_summary": reflection['conversation_summary'],
        "what_worked": reflection['what_worked'],
        "what_to_avoid": reflection['what_to_avoid'],
    })
add_episodic_memory(messages, client)


In [ ]:
def episodic_recall(query, vdb_client):

    # Load Database Collection
    episodic_memory = vdb_client.collections.get("episodic_memory")

    # Hybrid Semantic/BM25 Retrieval
    memory = episodic_memory.query.hybrid(
        query=query,
        alpha=0.5,
        limit=1,
    )

    return memory

query = "Talking about my name"

memory = episodic_recall(query, client)

memory.objects[0].properties

{'what_worked': "Providing specific paper recommendations and a concise summary to guide the user's research.",
 'conversation_summary': 'Recommended and summarized a key paper on the impact of climate change on coral reefs for a beginner in marine biology.',
 'context_tags': ['marine_biology', 'climate_change', 'coral_reefs'],
 'conversation': 'HUMAN: Hi good night\nAI: Good night! How can I assist you today?\nHUMAN: I want to learn more about marine biology, can you point me to good papers to start investigating about coral reef and the effect of global warming on it?\nAI: Certainly! A good starting point is the paper "Coral Reefs Under Rapid Climate Change and Ocean Acidification" by Hoegh-Guldberg et al., published in Science, and "Climate Change, Human Impacts, and the Resilience of Coral Reefs" by Hughes et al., also in Science.\nHUMAN: can you summarise in 100 words the first paper\nAI: The paper "Coral Reefs Under Rapid Climate Change and Ocean Acidification" by Hoegh-Guldberg 

In [ ]:
def episodic_system_prompt(query, vdb_client):
    # Get new memory
    memory = episodic_recall(query, vdb_client)

    current_conversation = memory.objects[0].properties['conversation']
    # Update memory stores, excluding current conversation from history
    if current_conversation not in conversations:
        conversations.append(current_conversation)
    # conversations.append(memory.objects[0].properties['conversation'])
    what_worked.update(memory.objects[0].properties['what_worked'].split('. '))
    what_to_avoid.update(memory.objects[0].properties['what_to_avoid'].split('. '))

    # Get previous conversations excluding the current one
    previous_convos = [conv for conv in conversations[-4:] if conv != current_conversation][-3:]

    # Create prompt with accumulated history
    episodic_prompt = f"""You are a helpful AI Assistant. Answer the user's questions to the best of your ability.
    You recall similar conversations with the user, here are the details:

    Current Conversation Match: {memory.objects[0].properties['conversation']}
    Previous Conversations: {' | '.join(previous_convos)}
    What has worked well: {' '.join(what_worked)}
    What to avoid: {' '.join(what_to_avoid)}

    Use these memories as context for your response to the user."""

    return SystemMessage(content=episodic_prompt)

In [ ]:
# Simple storage for accumulated memories
conversations = []
what_worked = set()
what_to_avoid = set()

# Start Storage for Historical Message History
messages = []

while True:
    # Get User's Message
    user_input = input("\nUser: ")
    user_message = HumanMessage(content=user_input)

    # Generate new system prompt
    system_prompt = episodic_system_prompt(user_input, client)

    # Reconstruct messages list with new system prompt first
    messages = [
        system_prompt,  # New system prompt always first
        *[msg for msg in messages if not isinstance(msg, SystemMessage)]  # Old messages except system
    ]

    if user_input.lower() == "exit":
        add_episodic_memory(messages, client)
        print("\n == Conversation Stored in Episodic Memory ==")
        break
    if user_input.lower() == "exit_quiet":
        print("\n == Conversation Exited ==")
        break

    # Add current user message
    messages.append(user_message)

    # Pass Entire Message Sequence to LLM to Generate Response
    response = llm.invoke(messages)
    print("\nAI Message: ", response.content)

    # Add AI's Response to Message List
    messages.append(response)


User: Hi

AI Message:  Hello! How can I assist you today?

User: I want to investigate about memory in AI

AI Message:  Investigating memory in AI is a fascinating topic! There are several aspects to consider, including how AI systems store and recall information, types of memory architectures, and how memory impacts learning and decision-making. Here are a few key concepts and resources that might be helpful:

1. **Neural Networks and Memory**: Traditional neural networks have limited memory capabilities. However, architectures like Recurrent Neural Networks (RNNs), Long Short-Term Memory networks (LSTMs), and Gated Recurrent Units (GRUs) are designed to handle sequences and remember information over time.

2. **Memory-Augmented Neural Networks (MANNs)**: These are neural networks endowed with an external memory component, such as Neural Turing Machines and Differentiable Neural Computers, which allow them to store and retrieve information in a more flexible and dynamic way.

3. **At

In [ ]:
from chunking_evaluation.chunking import RecursiveTokenChunker
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Papers/CoALA_Paper.pdf")
pages = loader.load_and_split()

# Combine all page contents into one string
document = " ".join(page.page_content for page in pages)

# Set up the chunker with your specified parameters
recursive_character_chunker = RecursiveTokenChunker(
    chunk_size=800,
    chunk_overlap=0,
    length_function=len,
    separators=["\n\n", "\n", ".", "?", "!", " ", ""]
)

# Split the combined text
recursive_character_chunks = recursive_character_chunker.split_text(document)

In [ ]:
len(recursive_character_chunks)

166

In [ ]:
client.collections.create(
    name="CoALA_Paper",
    description="Collection containing split chunks from the CoALA Paper",
    vectorizer_config=Configure.Vectorizer.text2vec_openai(),
    properties=[
        Property(name="chunk", data_type=DataType.TEXT),
    ]
)

In [ ]:
# Load Database Collection
coala_collection = client.collections.get("CoALA_Paper")

for chunk in recursive_character_chunks:
    # Insert Entry Into Collection
    coala_collection.data.insert({
        "chunk": chunk,
    })

In [ ]:
def semantic_recall(query, vdb_client):

    # Load Database Collection
    coala_collection = vdb_client.collections.get("CoALA_Paper")

    # Hybrid Semantic/BM25 Retrieval
    memories = coala_collection.query.hybrid(
        query=query,
        alpha=0.5,
        limit=15,
    )

    combined_text = ""

    for i, memory in enumerate(memories.objects):
        # Add chunk separator except for first chunk        if i > 0:


        # Add chunk number and content
        combined_text += f"\nCHUNK {i+1}:\n"
        combined_text += memory.properties['chunk'].strip()

    return combined_text

In [ ]:
memories = semantic_recall("What are the four kinds of memory", client)

print(memories)


CHUNK 1:
the next decision cycle, Voyager reasons over the environmental feedback to determine task completion. If
successful, Voyager selects a learning action adding the grounding procedure to procedural memory; otherwise,
it uses reasoning to refine the code and re-executes it. The importance of long-term memory and procedural
learning is empirically verified by comparing to baselines like ReAct and AutoGPT and ablations without the
procedural memory. Voyager is shown to better explore areas, master the tech tree, and zero-shot generalize
to unseen tasks.
Generative Agents(Park et al., 2023) are language agents grounded to a sandbox game affording interaction
with the environment and other agents. Its action space also has all four kinds of actions: grounding, reasoning,
CHUNK 2:
grounds to perception via the learned value function, Voyager’s grounding is text-only. It has a long-term
procedural memory that stores a library of code-based grounding procedures a.k.a.skills (e.g., “co

In [ ]:
def semantic_rag(query, vdb_client):

    memories = semantic_recall(query, vdb_client)

    semantic_prompt = f""" If needed, Use this grounded context to factually answer the next question.
    Let me know if you do not have enough information or context to answer a question.

    {memories}
    """

    return HumanMessage(semantic_prompt)

In [ ]:
message = semantic_rag("What are the four kinds of memory", client)
print(message)

content=' If needed, Use this grounded context to factually answer the next question.\n    Let me know if you do not have enough information or context to answer a question.\n    \n    \nCHUNK 1:\nthe next decision cycle, Voyager reasons over the environmental feedback to determine task completion. If\nsuccessful, Voyager selects a learning action adding the grounding procedure to procedural memory; otherwise,\nit uses reasoning to refine the code and re-executes it. The importance of long-term memory and procedural\nlearning is empirically verified by comparing to baselines like ReAct and AutoGPT and ablations without the\nprocedural memory. Voyager is shown to better explore areas, master the tech tree, and zero-shot generalize\nto unseen tasks.\nGenerative Agents(Park et al., 2023) are language agents grounded to a sandbox game affording interaction\nwith the environment and other agents. Its action space also has all four kinds of actions: grounding, reasoning,\nCHUNK 2:\ngrounds t

In [135]:
# Simple storage for accumulated memories
conversations = []
what_worked = set()
what_to_avoid = set()

# Start Storage for Historical Message History
messages = []

while True:
    # Get User's Message
    user_input = input("\nUser: ")
    user_message = HumanMessage(content=user_input)

    # Generate new system prompt
    system_prompt = episodic_system_prompt(user_input, client)

    # Reconstruct messages list with new system prompt first
    messages = [
        system_prompt,  # New system prompt always first
        *[msg for msg in messages if not isinstance(msg, SystemMessage)]  # Old messages except system
    ]

    if user_input.lower() == "exit":
        add_episodic_memory(messages, client)
        print("\n == Conversation Stored in Episodic Memory ==")
        break
    if user_input.lower() == "exit_quiet":
        print("\n == Conversation Exited ==")
        break

    # Get context and add it as a temporary message
    context_message = semantic_rag(user_input, client)

    # Pass messages + context + user input to LLM
    response = llm.invoke([*messages, context_message, user_message])
    print("\nAI Message: ", response.content)

    # Add only the user message and response to permanent history
    messages.extend([user_message, response])


User: What is the difference between empathy and sympathy? 

AI Message:  Empathy and sympathy are both ways of relating to the emotions and experiences of others, but they differ in how they connect to those feelings:

1. **Empathy** involves understanding and sharing the feelings of another person. When you empathize with someone, you put yourself in their shoes and try to experience their emotions from their perspective. Empathy is about feeling with someone and involves a deeper emotional connection.

2. **Sympathy**, on the other hand, involves recognizing someone else's emotional distress and feeling compassion or pity for them, but without necessarily sharing their emotional experience. Sympathy is more about acknowledging another person's feelings and offering comfort or support from a more detached standpoint.

In summary, empathy is about experiencing someone else's feelings as if they were your own, while sympathy is about acknowledging and caring about someone else's feeli

In [136]:
print(format_conversation(messages))


HUMAN: What is the difference between empathy and sympathy? 
AI: Empathy and sympathy are both ways of relating to the emotions and experiences of others, but they differ in how they connect to those feelings:

1. **Empathy** involves understanding and sharing the feelings of another person. When you empathize with someone, you put yourself in their shoes and try to experience their emotions from their perspective. Empathy is about feeling with someone and involves a deeper emotional connection.

2. **Sympathy**, on the other hand, involves recognizing someone else's emotional distress and feeling compassion or pity for them, but without necessarily sharing their emotional experience. Sympathy is more about acknowledging another person's feelings and offering comfort or support from a more detached standpoint.

In summary, empathy is about experiencing someone else's feelings as if they were your own, while sympathy is about acknowledging and caring about someone else's feelings withou

In [137]:
print(context_message.content)


 If needed, Use this grounded context to factually answer the next question.
    Let me know if you do not have enough information or context to answer a question.
    
    
CHUNK 1:
approach projects images directly into the language model’s representation space (Bavishi et al., 2023; Elsen
et al., 2023; Liu et al., 2023b). Integrated, multimodal reasoning may allow for more human-like behaviors: a
VLM-based agent could “see” a webpage, whereas a LLM-based agent would more likely be given raw HTML.
However, coupling the agent’s perception and reasoning systems makes the agent more domain-specific and
difficult to update. In either case, the basic architectural principles described by CoALA — internal memories,
a structured action space, and generalized decision-making — can be used to guide agent design.
Internal vs. external: what is the boundary between an agent and its environment?While
CHUNK 2:
people (Nguyen et al., 2021; Sumers et al., 2022; 2021; Wang et al., 2016). Agents capa

In [143]:
def episodic_system_prompt(query, vdb_client):
    # Get new memory
    memory = episodic_recall(query, vdb_client)

    # Load Existing Procedural Memory Instructions
    with open("./procedural_memory.txt", "r") as content:
        procedural_memory = content.read()

    # Get current conversation
    current_conversation = memory.objects[0].properties['conversation']

    # Update memory stores, excluding current conversation from history
    if current_conversation not in conversations:
        conversations.append(current_conversation)
    what_worked.update(memory.objects[0].properties['what_worked'].split('. '))
    what_to_avoid.update(memory.objects[0].properties['what_to_avoid'].split('. '))

    # Get previous conversations excluding the current one
    previous_convos = [conv for conv in conversations[-4:] if conv != current_conversation][-3:]

    # Create prompt with accumulated history
    episodic_prompt = f"""You are a helpful AI Assistant. Answer the user's questions to the best of your ability.
    You recall similar conversations with the user, here are the details:

    Current Conversation Match: {current_conversation}
    Previous Conversations: {' | '.join(previous_convos)}
    What has worked well: {' '.join(what_worked)}
    What to avoid: {' '.join(what_to_avoid)}

    Use these memories as context for your response to the user.

    Additionally, here are 10 guidelines for interactions with the current user: {procedural_memory}"""

    return SystemMessage(content=episodic_prompt)

In [146]:
def procedural_memory_update(what_worked, what_to_avoid):

    # Load Existing Procedural Memory Instructions
    with open("procedural_memory.txt", "r") as content:
        current_takeaways = content.read()

    # Load Existing and Gathered Feedback into Prompt
    procedural_prompt = f"""You are maintaining a continuously updated list of the most important procedural behavior instructions for an AI assistant. Your task is to refine and improve a list of key takeaways based on new conversation feedback while maintaining the most valuable existing insights.

    CURRENT TAKEAWAYS:
    {current_takeaways}

    NEW FEEDBACK:
    What Worked Well:
    {what_worked}

    What To Avoid:
    {what_to_avoid}

    Please generate an updated list of up to 10 key takeaways that combines:
    1. The most valuable insights from the current takeaways
    2. New learnings from the recent feedback
    3. Any synthesized insights combining multiple learnings

    Requirements for each takeaway:
    - Must be specific and actionable
    - Should address a distinct aspect of behavior
    - Include a clear rationale
    - Written in imperative form (e.g., "Maintain conversation context by...")

    Format each takeaway as:
    [#]. [Instruction] - [Brief rationale]

    The final list should:
    - Be ordered by importance/impact
    - Cover a diverse range of interaction aspects
    - Focus on concrete behaviors rather than abstract principles
    - Preserve particularly valuable existing takeaways
    - Incorporate new insights when they provide meaningful improvements

    Return up to but no more than 10 takeaways, replacing or combining existing ones as needed to maintain the most effective set of guidelines.
    Return only the list, no preamble or explanation.
    """

    # Generate New Procedural Memory
    procedural_memory = llm.invoke(procedural_prompt)

    # Write to File
    with open("./procedural_memory.txt", "w") as content:
        content.write(procedural_memory.content)

    return

prompt = procedural_memory_update(what_worked, what_to_avoid)

In [147]:
# Simple storage for accumulated memories
conversations = []
what_worked = set()
what_to_avoid = set()

# Start Storage for Historical Message History
messages = []

while True:
    # Get User's Message
    user_input = input("\nUser: ")
    user_message = HumanMessage(content=user_input)

    # Generate new system prompt
    system_prompt = episodic_system_prompt(user_input, client)

    # Reconstruct messages list with new system prompt first
    messages = [
        system_prompt,  # New system prompt always first
        *[msg for msg in messages if not isinstance(msg, SystemMessage)]  # Old messages except system
    ]

    if user_input.lower() == "exit":
        add_episodic_memory(messages, client)
        print("\n == Conversation Stored in Episodic Memory ==")
        procedural_memory_update(what_worked, what_to_avoid)
        print("\n== Procedural Memory Updated ==")
        break
    if user_input.lower() == "exit_quiet":
        print("\n == Conversation Exited ==")
        break

    # Get context and add it as a temporary message
    context_message = semantic_rag(user_input, client)

    # Pass messages + context + user input to LLM
    response = llm.invoke([*messages, context_message, user_message])
    print("\nAI Message: ", response.content)

    # Add only the user message and response to permanent history
    messages.extend([user_message, response])


User: Hi

AI Message:  Hello Juan! How can I assist you today?

User: I need you to remember my schedules can you do that?

AI Message:  I'm sorry, Juan, but I can't remember personal schedules or store personal information. However, I can help you set up reminders or organize your schedule using available tools. Let me know how I can assist you with that!

User: Can you set up a reminder for 3 pm on 01-12 to do homework?

AI Message:  I'm unable to set reminders directly. However, I can guide you on how to set a reminder using your device or a specific application. Would you like instructions for setting a reminder on a particular platform or device?

User: exit

 == Conversation Stored in Episodic Memory ==

== Procedural Memory Updated ==
